In [1]:
import requests
import json
import pandas as pd
import os
import dotenv
dotenv.load_dotenv()
FDAPI = os.environ["FUND_DAILY_API_KEY"]
FFAPI = os.environ["FUND_FACT_API_KEY"]
FundDailyKey = {
    'Ocp-Apim-Subscription-Key': FDAPI
}
FundFactKey = {
    'Ocp-Apim-Subscription-Key': FFAPI
}
project_type_keys = ['proj_retail_type', 'proj_term_flag', 'proj_term_dd', 'proj_term_mm','proj_term_yy']
specification_keys = ['spec_code', 'spec_desc']
suitability_keys = ['risk_spectrum', 'risk_spectrum_desc','fund_suitable_desc']

In [2]:
def FundFactAPIGet(url, proj_unique_id, type, key):
    if type == 'amc':
        response = requests.get(url,headers=key)
        if response.status_code == 200:
            return json.loads(response.text)
    elif type in ['project_type', 'specification', 'suitability']:
        response = requests.get(url + proj_unique_id + '/' + type, headers=key)
        if response.status_code == 200:
            return {'json': json.loads(response.text), 'status_code': response.status_code}
        else:
            dummy_json = {key: None for key in project_type_keys} if type == 'project_type' else ({key: None for key in specification_keys} if type == 'specification' else {key: None for key in suitability_keys})
            return {'json': dummy_json, 'status_code': response.status_code}

In [3]:
response = FundFactAPIGet('https://api.sec.or.th/FundFactsheet/fund/amc',None, 'amc', FundFactKey)
all_amc = pd.DataFrame(response)

In [4]:
def getFirstAMCName(row):
    name = row['name_en'].split()
    return name[0]

# Function to drop all df if proj_retail_type != R and write to new .CSV

In [5]:
def cleanAMCData(row):
    try:
        name = getFirstAMCName(row)
        df = pd.read_csv(f'../Result_amc/{name}/{name}_funds.csv')
        df = df[df['proj_retail_type'] == 'R']
        df.to_csv(f'../Result_amc/{name}/{name}_funds_clean.csv')
    except Exception as e:
        pass

In [ ]:
all_amc.apply(lambda row: cleanAMCData(row), axis=1)